In [1]:
!pip install efficientnet_pytorch
!pip install rembg
!pip install telebot
!pip install tqdm
!git clone https://github.com/ardipazij/fruits_dataset.git

  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━

In [4]:
import telebot
from PIL import Image
from io import BytesIO
import requests
from torchvision import datasets, transforms
from inference_model import ModelInference
import os

In [5]:
!rm -r /content/FruitsVegetables/.git
!rm -r /content/FruitsVegetables/*.md

In [9]:
data_dir = '/content/FruitsVegetables'

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
train_data = datasets.ImageFolder(os.path.join(data_dir, 'Training'), transform=train_transforms)

In [ ]:
TOKEN = 'TOKEN'

MODEL_PATH = "epoch_5.pth"

bot = telebot.TeleBot(TOKEN)

model_inference = ModelInference(model_path=MODEL_PATH, classes=train_data.classes)

@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "Привет! Отправь мне изображение, и я скажу тебе, что это.")

@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    try:
        file_info = bot.get_file(message.photo[-1].file_id)
        downloaded_file = bot.download_file(file_info.file_path)

        with open('current_image.jpg', 'wb') as new_file:
            new_file.write(downloaded_file)

        prediction = model_inference.inference('current_image.jpg')
        bot.reply_to(message, f'Я думаю, это {prediction}.')
    except Exception as e:
        bot.reply_to(message, f'Произошла ошибка: {e}')

if __name__ == '__main__':
    bot.polling(none_stop=True)